In [1]:
!pip install catalyst
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install efficientnet_pytorch
!pip install timm
!pip install torchtoolbox

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-302a11_v
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-302a11_v
  Resolved https://github.com/albu/albumentations to commit 28c6049e498d326283a6a6d49dae3e22041ba456
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=b4849959ab9b16879dce0cdb632cefa29051a0f00b379761b1bfff37f7dab897
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 376 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 1.7 MB/s eta 0:00:011


In [2]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations
from albumentations import pytorch as AT
import torch
import torch.nn as nn

from efficientnet_pytorch import EfficientNet
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from albumentations.core.composition import Compose, OneOf
from albumentations.augmentations.transforms import CLAHE, GaussNoise, ISONoise
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchtoolbox.transform import ImageNetPolicy
import matplotlib.pyplot as plt
img_size = 224

In [3]:
class CFG:
    seed = 42
    model_name = 'efficientnet-b3'
    pretrained = True
    img_size = 224
    num_classes = 7
    lr = 3e-4
    min_lr = 1e-6
    t_max = 20
    num_epochs = 40
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [4]:
class Dataset(BaseDataset):
    def __init__(
            self, 
            data_dir=None,
            augmentation=None, 
    ):
        # self.ids = os.listdir(images_dir)
        # self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        # self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # self.ids = []
        self.images = []
        for emo_dir in os.listdir(data_dir):
            cur_dir = os.path.join(data_dir, emo_dir)
            for file in os.listdir(cur_dir):
                self.images.append((emo_dir, os.path.join(cur_dir, file)))
#         try:
    #         except:
#                 print(data_fer_dir)
        labels_tr = ['empty', 'full']
        self.map_labels = {name : ind for ind, name in enumerate(labels_tr)}
        self.augmentation = augmentation
    
    def get_images_from_folder(self, path):
        return os.listdir(path)

    def __getitem__(self, i):
        
        # read data
        cur_pair = self.images[i]
        image = cv2.imread(cur_pair[1])
#         if 'train_fer' not in cur_pair[1]:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         else:
#             image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.make_square(image)
        label = cur_pair[0]

        # apply augmentations
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        
        return image, self.map_labels[label]
    

    def __len__(self):
        return len(self.images)
    def make_square(self, img):
        sz = max(img.shape[0], img.shape[1])
        left = (sz - img.shape[1]) // 2
        right = sz - img.shape[1] - left
        top = (sz - img.shape[0]) // 2
        bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
        return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [5]:
def make_square(img):
    sz = max(img.shape[0], img.shape[1])
    left = (sz - img.shape[1]) // 2
    right = sz - img.shape[1] - left
    top = (sz - img.shape[0]) // 2
    bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
    return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [6]:


data_transform = {
    'val': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'train': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.HorizontalFlip(0.5),
        albumentations.RGBShift(),
        albumentations.Blur(),
        albumentations.GaussNoise(),
#         albumentations.ElasticTransform(),
        albumentations.CoarseDropout(p=1),
        albumentations.Rotate(always_apply=False, p=1.0, limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
#         albumentations.ShiftScaleRotate(p=0.5),
#         albumentations.RandomBrightnessContrast(p=0.5),
#         ImageNetPolicy,
        
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'test': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ])
}



In [7]:
# helper function for data visualization
def visualize(images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    cur = 0
    for i, (image, name) in enumerate(images):
        plt.title(name)
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image.numpy().transpose(1, 2, 0))
        cur += 1
            
    plt.show()

In [8]:
class MyEffnet(nn.Module):
    def __init__(self, model_name='efficientnet-b3'):
        super().__init__()
        self.model = EfficientNet.from_pretrained(model_name, advprop=True)
#         in_features = self.model.get_classifier().in_features
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, self.model._fc.in_features // 2),
            nn.ReLU(),
            nn.Linear(self.model._fc.in_features // 2, 2),
        )
    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
model = MyEffnet()
# model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# model.to("cuda")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/adv-efficientnet-b3-cdd7c0f4.pth" to /root/.cache/torch/hub/checkpoints/adv-efficientnet-b3-cdd7c0f4.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [11]:
# from catalyst.contrib.schedulers import MultiStepLR, ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
train_data = Dataset(data_dir='../input/trash-dataset-class/train_test/train', augmentation=data_transform['train'])
valid_data = Dataset(data_dir='../input/trash-dataset-class/train_test/test', augmentation=data_transform['test'])
loaders = {
    "train": DataLoader(train_data, batch_size=32, shuffle=True),
    "valid": DataLoader(valid_data, batch_size=32),
}

In [ ]:
!nvidia-smi

In [ ]:
# optimizer.param_groups[0]['lr'] = 1e-5

In [12]:
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    num_epochs=20,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=[1]),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=2
        ),
        dl.SchedulerCallback(loader_key="valid", metric_key="loss"),
        dl.CheckpointCallback(
            logdir="./logs", loader_key="valid", metric_key="loss", minimize=True
        ),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
#     minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)


1/20 * Epoch (train):   0%|          | 0/29 [00:00<?, ?it/s]

train (1/20) accuracy: 0.8228941696251444 | accuracy/std: 0.15412663727531198 | accuracy01: 0.8228941696251444 | accuracy01/std: 0.15412663727531198 | f1/_macro: 0.8228065132456189 | f1/_micro: 0.8228891684969031 | f1/_weighted: 0.8229304969976038 | f1/class_00: 0.8189795481010216 | f1/class_01: 0.8266334783902163 | loss: 0.3898903770657904 | loss/mean: 0.3898903770657904 | loss/std: 0.19655239215693612 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.8227568767961781 | precision/_micro: 0.8228941684665227 | precision/_weighted: 0.8231687518072118 | precision/class_00: 0.8100436681222707 | precision/class_01: 0.8354700854700855 | recall/_macro: 0.8230583158995816 | recall/_micro: 0.8228941684665227 | recall/_weighted: 0.8228941684665227 | recall/class_00: 0.828125 | recall/class_01: 0.8179916317991631 | support/class_00: 448.0 | support/class_01: 478.0


1/20 * Epoch (valid):   0%|          | 0/4 [00:00<?, ?it/s]

valid (1/20) accuracy: 0.8103448275862069 | accuracy/std: 0.20086954899173723 | accuracy01: 0.8103448275862069 | accuracy01/std: 0.20086954899173723 | f1/_macro: 0.7986063257424562 | f1/_micro: 0.8103398276170577 | f1/_weighted: 0.801120688494861 | f1/class_00: 0.8472173389556157 | f1/class_01: 0.7499953125292966 | loss: 0.5684486997538599 | loss/mean: 0.5684486997538599 | loss/std: 0.4922412666321962 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.8674698795180723 | precision/_micro: 0.8103448275862069 | precision/_weighted: 0.8606148732862484 | precision/class_00: 0.7349397590361446 | precision/class_01: 1.0 | recall/_macro: 0.8 | recall/_micro: 0.8103448275862069 | recall/_weighted: 0.8103448275862069 | recall/class_00: 1.0 | recall/class_01: 0.6 | support/class_00: 61.0 | support/class_01: 55.0
* Epoch (1/20) lr: 0.0003 | momentum: 0.9


2/20 * Epoch (train):   0%|          | 0/29 [00:00<?, ?it/s]

...

valid (19/20) accuracy: 0.9568965496688053 | accuracy/std: 0.013825861686873553 | accuracy01: 0.9568965496688053 | accuracy01/std: 0.013825861686873553 | f1/_macro: 0.9567340173332457 | f1/_micro: 0.9568915517502639 | f1/_weighted: 0.9568690471532062 | f1/class_00: 0.9593445938524817 | f1/class_01: 0.9541234408140098 | loss: 0.2746496575659719 | loss/mean: 0.2746496575659719 | loss/std: 0.12680544712393027 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.9572879330943848 | precision/_micro: 0.9568965517241379 | precision/_weighted: 0.9569943970666996 | precision/class_00: 0.9516129032258065 | precision/class_01: 0.962962962962963 | recall/_macro: 0.9563338301043218 | recall/_micro: 0.9568965517241379 | recall/_weighted: 0.9568965517241379 | recall/class_00: 0.9672131147540983 | recall/class_01: 0.9454545454545454 | support/class_00: 61.0 | support/class_01: 55.0
* Epoch (19/20) lr: 2.9999999999999997e-05 | momentum: 0.9


20/20 * Epoch (train):   0%|          | 0/29 [00:00<?, ?it/s]

train (20/20) accuracy: 0.9956803455723542 | accuracy/std: 0.010791457196365596 | accuracy01: 0.9956803455723542 | accuracy01/std: 0.010791457196365596 | f1/_macro: 0.9956708069587605 | f1/_micro: 0.9956753455974625 | f1/_weighted: 0.9956753455974626 | f1/class_00: 0.9955307143108263 | f1/class_01: 0.9958108996066949 | loss: 0.01492231088030689 | loss/mean: 0.01492231088030689 | loss/std: 0.015023183370598705 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.9956758069336522 | precision/_micro: 0.9956803455723542 | precision/_weighted: 0.9956803455723542 | precision/class_00: 0.9955357142857143 | precision/class_01: 0.99581589958159 | recall/_macro: 0.9956758069336522 | recall/_micro: 0.9956803455723542 | recall/_weighted: 0.9956803455723542 | recall/class_00: 0.9955357142857143 | recall/class_01: 0.99581589958159 | support/class_00: 448.0 | support/class_01: 478.0


20/20 * Epoch (valid):   0%|          | 0/4 [00:00<?, ?it/s]

valid (20/20) accuracy: 0.9568965496688053 | accuracy/std: 0.013825861686873553 | accuracy01: 0.9568965496688053 | accuracy01/std: 0.013825861686873553 | f1/_macro: 0.9567340173332457 | f1/_micro: 0.9568915517502639 | f1/_weighted: 0.9568690471532062 | f1/class_00: 0.9593445938524817 | f1/class_01: 0.9541234408140098 | loss: 0.27782007589422425 | loss/mean: 0.27782007589422425 | loss/std: 0.12640602999457906 | lr: 2.9999999999999997e-05 | momentum: 0.9 | precision/_macro: 0.9572879330943848 | precision/_micro: 0.9568965517241379 | precision/_weighted: 0.9569943970666996 | precision/class_00: 0.9516129032258065 | precision/class_01: 0.962962962962963 | recall/_macro: 0.9563338301043218 | recall/_micro: 0.9568965517241379 | recall/_weighted: 0.9568965517241379 | recall/class_00: 0.9672131147540983 | recall/class_01: 0.9454545454545454 | support/class_00: 61.0 | support/class_01: 55.0
* Epoch (20/20) lr: 2.9999999999999997e-05 | momentum: 0.9
Top best models:
logs/train.5.pth	0.1580


In [ ]:
# mod = runner.bes

In [ ]:
# import os
# os.listdir('../input/emotion-recognition-dataset/RDFKDEF_yolo_bounding_box30px/rfd_cropped/test_many_photos_per_human')

In [ ]:
# print(torch.load('./logs/train.7.pth'))

In [ ]:
# pretrained_model = MyEffnet()
# # pretrained_model = torch.load('./logs/train.7.pth')
# pretrained_model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# pretrained_model.to('cuda')
# pretrained_model.eval()

In [ ]:
# model.cuda()
# model = EmotionsLightning(MyEffnet())
# model = EmotionsLightning(model=MyEffnet()).load_from_checkpoint(checkpoint_callback.best_model_path, model=MyEffnet())
# model.to('cuda')
# model = MyEffnet()
# model_cfg = torch.load(checkpoint_callback.best_model_path)
# checkpoint_callback.best_model_path
# model.eval()
# model = MyEffnet()
# model.load_state_dict(checkpoint_callback.best_model_path, model_cfg['state_dict'])
# model_cfg['state_dict']
# # print(model)
# model.to('cuda')

In [ ]:
# model.load_state_dict(utils.get_averaged_weights_by_path_mask(logdir="./logs", path_mask="*.pth"))

In [13]:
torch.save(model, 'best_model.pth')
torch.save(model.state_dict(), 'best_model_dict.pth')